In [1]:
import os
import cv2
import shutil
from ultralytics import YOLO

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="RXuXsSYOEKui6hojXnVd")
project = rf.workspace("david-lee-d0rhs").project("american-sign-language-letters")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to American-Sign-Language-Letters-1 in yolov8:: 100%|██████████| 3468/3468 [00:02<00:00, 1413.95it/s]


Move dataset to new file "ASL" with below command from terminal
```python
mv American* ASL
```

In [3]:
def apply_clahe_and_copy_files(source_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(source_dir):
        relative_path = os.path.relpath(root, source_dir)
        output_subdir = os.path.join(output_dir, relative_path)

        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        for filename in files:
            input_path = os.path.join(root, filename)
            output_path = os.path.join(output_subdir, filename)

            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                try:
                    # Apply CLAHE to images
                    image = cv2.imread(input_path, cv2.IMREAD_COLOR)
                    if image is not None:
                        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
                        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
                        l, a, b = cv2.split(lab)
                        l = clahe.apply(l)
                        lab = cv2.merge((l, a, b))
                        output_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
                        cv2.imwrite(output_path, output_image)
                    else:
                        print(f"Unable to process {input_path}")
                except Exception as e:
                    print(f"Error processing {input_path}: {str(e)}")

            else:
                try:
                    # Copy other files (non-image files)
                    shutil.copy(input_path, output_path)
                except Exception as e:
                    print(f"Error copying {input_path} to {output_path}: {str(e)}")


apply_clahe_and_copy_files('ASL', 'CLAHED')


In [ ]:
model = YOLO('yolov8n.pt')

model.train(data='CLAHED/data.yaml', epochs=100, patience=25, imgsz=640, close_mosaic=20, optimizer='auto', pretrained = True)

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/CLAHED/data.yaml, epochs=100, patience=25, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torch

100%|██████████| 755k/755k [00:00<00:00, 22.1MB/s]


Overriding model.yaml nc=80 with nc=26

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/CLAHED/train/labels... 1512 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1512/1512 [00:00<00:00, 1746.07it/s]


train: New cache created: /content/CLAHED/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

     55/100      2.39G     0.5731     0.6636      1.059         23        640: 100%|██████████| 95/95 [00:30<00:00,  3.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.20it/s]


                   all        144        144      0.915      0.906      0.948      0.787
Stopping training early as no improvement observed in last 25 epochs. Best results observed at epoch 30, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

55 epochs completed in 0.503 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3010718 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.12it/s]


                   all        144        144      0.853      0.914      0.951      0.789
                     A        144          5      0.902        0.8       0.92       0.77
                     B        144          9          1      0.788      0.995      0.867
                     C        144          3      0.893          1      0.995      0.895
                     D        144          6      0.839      0.876      0.972      0.772
                     E        144          4      0.876          1      0.995      0.822
                     F        144          8      0.782          1      0.995      0.862
                     G        144          5      0.846          1      0.995      0.814
                     H        144          9          1       0.87      0.995      0.743
                     I        144          2      0.761        0.5       0.52      0.466
                     J        144          8      0.872          1      0.995      0.692
                     

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f8f8a8d38e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [ ]:
metrics = model.val(split = 'test')

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3010718 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/CLAHED/test/labels... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<00:00, 393.61it/s]

val: New cache created: /content/CLAHED/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.64it/s]


                   all         72         72      0.828      0.867      0.913      0.763
                     A         72          1      0.711          1      0.995      0.995
                     B         72          3        0.9          1      0.995      0.809
                     C         72          4      0.956          1      0.995      0.779
                     D         72          1      0.733          1      0.995      0.895
                     F         72          2      0.837          1      0.995      0.821
                     G         72          5      0.918          1      0.995      0.755
                     H         72          3      0.715      0.858       0.83        0.6
                     I         72          2      0.722        0.5      0.502      0.502
                     J         72          4      0.892          1      0.995      0.723
                     K         72          4          1      0.908      0.995      0.895
                     